In [1]:

# 주의사항, 문자열 안에 특수문자 부분 체크 잘하기.
# create table newsTable (
# id int not null auto_increment,
# title varchar(200),
# publisher varchar(200),
# newsDate varchar(10),
# newsTime varchar(10),
# newsDetail varchar(200),
# newsImgUrl varchar(200),
# primary key(id)
# );
# regex 이용해서, re.sub(매개변수1,매개변수2,매개변수3)
# 매개변수1 : 특정 조건
# 매개변수2 : 대체할 문자
# 매개변수3 : regex를 적용할 문장.


In [2]:
import time
import bs4
import urllib.request
import pymysql
import re
def clean_text(inputString):
    text_rmv = re.sub(
        '[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inputString)
    return text_rmv

# 전역변수, 기사에서 가지고 올 정보만 선택한 변수.
imgLinkUrl, subject, link, press, pDate, pTime = "", "", "", "", "", ""
def insertData(subject, press, pDate, pTime, link, imgLinkUrl):
    # con : 파이썬 <-> mysql 연결하기 위한 인스턴스, 도구
    # cur : 디비의 정보를 조작하기위한 도구.
    con, cur = None, None
    data = ""
    data0, data1, data2, data3, data4, data5, data6 = "", "", "", "", "", "", ""
    sql = ""
    con = pymysql.connect(host='127.0.0.1', user='webuser',
                          password='webuser', database='pydb', charset='utf8')
    cur = con.cursor()
    
    data1 = subject
    data2 = press
    data3 = pDate
    data4 = pTime
    data5 = link
    data6 = imgLinkUrl
    
    try:
        # print(null)
        # print("데이터 실행전 확인")
        # print(data1)
        # print(data2)
        # print(data3)
        # print(data4)
        # print(data5)
        # print(data6)
        query = "INSERT INTO newsTable (title, publisher, newsDate, newsTime, newsDetail, newsImgUrl) VALUES (%s, %s, %s, %s, %s, %s)"
        values = (data1, data2, data3, data4, data5, data6)
        print("순서2 : sql 실행전 ")
        # print(f"sql : {sql}")

        cur.execute(query, values)

    except:
        print('================================')
        print("순서3 : 예외 발생")
        print('================================')
        
    else:
        print('================================')
        print("순서3: 성공")
        print('================================')
        
    con.commit()
    con.close()
##
page = 1
count = 1
nateUrl = "https://news.nate.com/recent?cate=its&mid=n0105&type=c&date=20240719&page=1"

while True:
  if (count != 201):
    try:
        # newsUrl = nateUrl + str(page)
        newsUrl = nateUrl
        # page += 1
        htmlObject = urllib.request.urlopen(newsUrl)
        webPage = htmlObject.read()
        bsObject = bs4.BeautifulSoup(webPage, 'html.parser')
        tag_list = bsObject.findAll('div', {'class': 'mlt01'})
        print('###### 실시간 뉴스 속보 #######')
        for tag in tag_list:
          if (count != 201):
            # strong -> h2 변경 231210 (확인 날짜)
            subject = tag.find('h2', {'class': 'tit'}).text
            subject = clean_text(subject)
            link = tag.find('a', {'class': 'lt1'})['href']
            link = 'https:' + link
            imgLink = tag.find('em', {'class': 'mediatype'})
            # 이미지 없는 경우 처리 부분. 파이썬 null 대신  None
            if imgLink != None:
                imgLinkUrl = imgLink.find('img')['src']
                imgLinkUrl = 'https:' + imgLinkUrl
            else:
                imgLinkUrl = '이미지가 존재 하지 않음'
            pressAndDate = tag.find('span', {'class': 'medium'}).text
            pressAndDate.replace('\t', ' ')
            pressAndDate.replace('\n', '')
            if len(pressAndDate.split()) == 3:
                press, pDate, pTime = pressAndDate.split()
            elif len(pressAndDate.split()) == 4:
                press1, press2, pDate, pTime = pressAndDate.split()
                press = press1+press2
            else:
                pTime = 'pTime없음'
                pDate = 'pDate없음'
                press = 'press없음'
                # continue
            print("순서1: 데이터 추가 전")
            print("subject: %s" % (subject))
            print("press: %s" % (press))
            print("pDate: %s" % (pDate))
            print("pTime: %s" % (pTime))
            print("link: %s" % (link))
            print("imgLinkUrl: %s" % (imgLinkUrl))
            print("count : %d" % count)
            print('================================')
            print('================================')
            insertData(subject, press, pDate, pTime, link, imgLinkUrl)
            count += 1
            # print('(' , page, ')', subject)
            # print('\t https:'+link, press, pDate, pTime)
            # print('\t imgLink : '+ imgLinkUrl)
          else:
            break
        
        time.sleep(1)
        print(f"count : {count}, page : {page}")
        # print(f"완료!")
        
    except:
        break
  else:
     print(f"완료!")
     break
    # else:
    #     break
    # 특정 시간마다 동작 여부
    # time.sleep(60)

<>:8: SyntaxWarning: invalid escape sequence '\?'
<>:8: SyntaxWarning: invalid escape sequence '\?'
C:\Users\it\AppData\Local\Temp\ipykernel_6180\1108745712.py:8: SyntaxWarning: invalid escape sequence '\?'
  '[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', ' ', inputString)


###### 실시간 뉴스 속보 #######
순서1: 데이터 추가 전
subject: 네이버 최수연  소버린 AI 확산 위해 각국 기업과 파트너십 구축할 것 
press: 헤럴드경제
pDate: 07-19
pTime: 14:28
link: https://news.nate.com/view/20240719n11381?mid=n0105
imgLinkUrl: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/hr/2024/07/19/20240719050437_0.jpg
count : 1
순서2 : sql 실행전 
순서3: 성공
순서1: 데이터 추가 전
subject: 아재폰이라니  2030 잇템된  폴더블폰  싸게 사는 법
press: 이데일리
pDate: 07-19
pTime: 14:28
link: https://news.nate.com/view/20240719n18872?mid=n0105
imgLinkUrl: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ed/2024/07/19/PS24071900607.jpg
count : 2
순서2 : sql 실행전 
순서3: 성공
순서1: 데이터 추가 전
subject: 넥슨 아이언메이스  다크앤다커  분쟁 9월10일 결판
press: 쿠키뉴스
pDate: 07-19
pTime: 14:27
link: https://news.nate.com/view/20240719n19739?mid=n0105
imgLinkUrl: https://thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/ck/2024/07/19/kuk202407190112.680x.0.jpg
count : 3
순서2 : sql 실행전 
순서3: 성공
순서1: 데이터 추가 전
subject: 유상임 후보  R&D 예산 반영 최선  전문성 부족 지적은  일축 
press: 아시